In [147]:
from IPython.display import display
from ipywidgets import widgets, interactive
%run filterTrailList.ipynb
%run scrapeAllTrails.ipynb
%run scrapeColorado14ers.ipynb
%run match14ers.ipynb
%run filter14ersbyClass.ipynb

layout = widgets.Layout(width='400px', height='20px')

street = widgets.Text(
    value='333 E Colorado Ave',
    placeholder='Type something',
    description='Street Address:',
    disabled=False,
    style= {'description_width': 'initial'},
)
city = widgets.Text(
    value='Colorado Springs',
    placeholder='Type something',
    description='City:',
    disabled=False
)
state = widgets.Text(
    value='CO',
    placeholder='Type something',
    description='State:',
    disabled=False
)
zip_code = widgets.Text(
    value='80903',
    placeholder='Type something',
    description='Zip Code:',
    disabled=False
)
display(street)
display(city)
display(state)
display(zip_code)



slider1 = widgets.FloatRangeSlider(
    min=0,
    max=30,
    step= 0.25,
    display='flex',
    description='Hiking Distance (miles):',
    style= {'description_width': 'initial'},
    layout = layout
)
display(slider1)

slider2 = widgets.FloatSlider(
    min=0,
    max=7,
    step= 0.25,
    description='Driving Distance (hours):',
    style= {'description_width': 'initial'},
    layout = layout,
    value = 3.0
)
display(slider2)
slider3 = widgets.FloatSlider(
    min=1,
    max=4,
    step= .5,
    description='Class Limit (for 14ers only):',
    style= {'description_width': 'initial'},
    layout = layout,
    value = 2
)
display(slider3)

feature1 = widgets.RadioButtons(
    options=['Lake','Wildflowers', 'River','Views','Dogs allowed', 'Scramble','Waterfall', 'Forest', 'Wildlife'],
    value='Wildflowers',
    description='Feature #1:',
)

feature2 = widgets.RadioButtons(
    options=['Lake','Wildflowers', 'River','Views','Dogs allowed', 'Scramble','Waterfall', 'Forest', 'Wildlife'],
    value='Lake',
    description='Feature #2:',
)
display(feature1)
display(feature2)

weekday = widgets.Dropdown(
    options=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
    value='Sunday',
    description='Hiking Day:',
)
display(dow)

hike_list = widgets.Dropdown(
    options=['All Colorado','Colorado short list','All 14ers','Top 14ers'],
    value='All Colorado',
    description='Trail List:',
)
display(hike_list)

feature_dict = {'Lake':'lake','Wildflowers':'wild-flowers', 'River':'river','Views':'views',\
                'Dogs allowed':'dogs','Scramble':'scramble','Waterfall':'waterfall', 'Forest':'forest', 'Wildlife':'wildlife'}

# feature_requests = [feature_dict[feature1.value], feature_dict[feature2.value]]

# address = street.value + ' '+ city.value + ', '+state.value +' '+ zip_code.value

# hike_day = dow.value
# dow = dow.value
# distance_max = slider1.value[1]
# distance_min = slider1.value[0]

# drive_lim = slider2.value
# class_limit = slider3.value

# trail_list = hike_list.value

# check_weather = True

output = widgets.Output()
done = widgets.Button(
    description='Done',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
output = widgets.Output()
display(done, output)

def on_button_clicked(b):
    with output:

        feature_requests = [feature_dict[feature1.value], feature_dict[feature2.value]]

        address = street.value + ' '+ city.value + ', '+state.value +' '+ zip_code.value

        hike_day = weekday.value
        dow = hike_day
        distance_max = slider1.value[1]
        distance_min = slider1.value[0]

        drive_lim = slider2.value
        class_limit = slider3.value

        trail_list = hike_list.value

        check_weather = True

        full_df = allTrailsAny(address, trail_list, hike_day)

        full_df_in= full_df[:][:]


        filtered_hikes_df = filterHikes(full_df_in[:][:],distance_max,distance_min,drive_lim,feature_requests,check_weather,dow)


        if trail_list == 'Top 14ers' or  trail_list == 'All 14ers':

            all14erData = get14erData()

            co_14_df = all14erData[:][:]
            at_df = filtered_hikes_df[:][:]

            merged_14er_data = get14erMatches(at_df,co_14_df)

            filtered_14er_df = filter14ers(merged_14er_data,class_limit)

            filtered_14er_df.to_csv('filtered_14ers_'+str(date.today())+'.csv')
        else:
            
            filtered_hikes_df.to_csv('filtered_hikes_'+str(date.today())+'.csv')

        
done.on_click(on_button_clicked)

Text(value='333 E Colorado Ave', description='Street Address:', placeholder='Type something', style=Descriptio…

Text(value='Colorado Springs', description='City:', placeholder='Type something')

Text(value='CO', description='State:', placeholder='Type something')

Text(value='80903', description='Zip Code:', placeholder='Type something')

FloatRangeSlider(value=(7.5, 22.5), description='Hiking Distance (miles):', layout=Layout(height='20px', width…

...

RadioButtons(description='Feature #2:', options=('Lake', 'Wildflowers', 'River', 'Views', 'Dogs allowed', 'Scr…

Dropdown(description='Hiking Day:', options=('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',…

Dropdown(description='Trail List:', options=('All Colorado', 'Colorado short list', 'All 14ers', 'Top 14ers'),…

Button(description='Done', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()